In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from torchvision import datasets, transforms

In [ ]:
!pip install unrar
!unrar x "/content/drive/MyDrive/dataset_classification.rar"

Выходные данные были обрезаны до нескольких последних строк (5000).
Extracting  dataset_classification/Кирпич/frame_243_46.jpg                80%  OK 
Extracting  dataset_classification/Кирпич/frame_243_47.jpg                80%  OK 
Extracting  dataset_classification/Кирпич/frame_243_48.jpg                80%  OK 
Extracting  dataset_classification/Кирпич/frame_243_49.jpg                80%  OK 
Extracting  dataset_classification/Кирпич/frame_243_5.jpg                 80%  OK 
Extracting  dataset_classification/Кирпич/frame_243_50.jpg                80%  OK 
Extracting  dataset_classification/Кирпич/frame_243_51.jpg                80%  OK 
Extracting  dataset_classification/Кирпич/frame_243_52.jpg                80%  OK 
Extracting  dataset_classification/Кирпич/frame_243_53.jpg                80%  OK 
Extracting  dataset_classification/Кирпич/frame_243_54.jpg                80%  OK 
Extracting  

In [ ]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

val_transforms = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]
)


test_transforms = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]
)

In [ ]:
dataset = datasets.ImageFolder("/content/dataset_classification")

In [ ]:
!pip -q install vit_pytorch linformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.2 MB/s eta 0:00:00


In [ ]:
from linformer import Linformer
from vit_pytorch.efficient import ViT
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm.notebook import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [71]:
# Training settings
batch_size = 64
epochs = 100
lr = 3e-5
gamma = 0.7
seed = 42

In [72]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)

In [73]:
model = ViT(
    dim=128,
    image_size=224,
    patch_size=32,
    num_classes=4,
    transformer=efficient_transformer,
    channels=3,
).to(device)

In [74]:
torch.manual_seed(27)
indices = torch.randperm(len(dataset)).tolist()

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])
train_dataset.dataset.transform = transforms.Lambda(lambda x: train_transforms(x))
val_dataset.dataset.transform = transforms.Lambda(lambda x: val_transforms(x))

data_loader_train = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=True, num_workers=4)
data_loader_val = torch.utils.data.DataLoader(
    val_dataset, batch_size=64, shuffle=False, num_workers=4)

In [75]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0, checkpoint_path='checkpoint.pth'):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.checkpoint_path = checkpoint_path

    def __call__(self, val_loss, model):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
            self.save_checkpoint(val_loss, model)
        else:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

    def save_checkpoint(self, val_loss, model):
        print(f'Validation loss decreased ({self.best_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.checkpoint_path)


In [76]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

early_stopping = EarlyStopping(patience=5, delta=0.001)

In [ ]:
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(data_loader_train):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(data_loader_train)
        epoch_loss += loss / len(data_loader_train)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in data_loader_val:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(data_loader_val)
            epoch_val_loss += val_loss / len(data_loader_val)

    early_stopping(epoch_val_loss, model)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

  0%|          | 0/252 [00:00<?, ?it/s]